In [29]:
# import library dependencies
import numpy as np 
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer # Used to convert textual to numerical values
from sklearn.metrics.pairwise import cosine_similarity # algoritma cosine similarity untuk recommended

In [49]:
# Import data dan data processing
# Loading data dari CSV dan convert ke dataframe
travels_data = pd.read_csv('./travel.csv')
travels_data 

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [50]:
# print first 5 dataframe
travels_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [51]:
# Mengecek ada berapa data rows dan colomn di data frame
travels_data.shape

(4803, 24)

In [52]:
# Mengambil relevan feature untuk recommendation 
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
selected_features

['genres', 'keywords', 'tagline', 'cast', 'director']

In [53]:
# replace null values menjadi null string
for feature in selected_features:
    travels_data[feature] = travels_data[feature].fillna('')

In [54]:
# cek lagi datanya udah keubah
travels_data

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,,http://shanghaicalling.com/,126186,,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en"", ""name""

In [55]:
# combining all 5 selected feature
combine_features = travels_data['genres'] + ' ' + travels_data['keywords'] + ' ' + travels_data['tagline'] + ' ' + travels_data['cast'] + ' ' + travels_data['director']
combine_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [56]:
# Convert text data to feature vectors
# Karena cosine_similarity harus pake vectors 
vectorizer = TfidfVectorizer()

# Merubah ke feature vectors dimana data menjadi numerical values
feature_vectors = vectorizer.fit_transform(combine_features)
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

In [57]:
# Menggunakan algoritma cosine similarity untuk recommended 
# Code ini akan mendapatkan score dari cosine similarity function

similarity = cosine_similarity(feature_vectors)
similarity

array([[1.        , 0.07219487, 0.037733  , ..., 0.        , 0.        ,
        0.        ],
       [0.07219487, 1.        , 0.03281499, ..., 0.03575545, 0.        ,
        0.        ],
       [0.037733  , 0.03281499, 1.        , ..., 0.        , 0.05389661,
        0.        ],
       ...,
       [0.        , 0.03575545, 0.        , ..., 1.        , 0.        ,
        0.02651502],
       [0.        , 0.        , 0.05389661, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02651502, 0.        ,
        1.        ]])

In [65]:
# Akan merepresentasikan index dan nilai values, jadi 1 index(travel) 
# akan mendapatkan nilai similarity dengan seluruh travel
print(similarity.shape)

(4803, 4803)


In [72]:
# Input favorite travel spot saat liburan
travel_favorite = input("Enter your favorite travel spot: ")


In [67]:
# Membuat list dengan seluruh travel name dari data set
list_of_all_travel_name = travels_data['title'].tolist()
list_of_all_travel_name

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [73]:
# Menemukan close match dari travel spot yang di input user
# Kita gunakan difflib disini

find_close_match = difflib.get_close_matches(travel_favorite, list_of_all_travel_name)
find_close_match

['Mi America', 'The American', 'American Pie']

In [74]:
# Mengambil close match paling dekat yaitu index 0
close_match = find_close_match[0]
close_match

'Mi America'

In [ ]:
# Menemukan index dari travel dengan travel spot
index_of_travel = travels_data[travels_data.title == close_match]['index'].values[0]
index_of_travel

In [75]:
# Mengambil list dari similar travel based on index of travel yang sudah dibuat
# akan menghasilkan index dan score nilainya berdasarkan similarity
similarity_score = list(enumerate(similarity[index_of_travel])) ## enumaret for loop data
similarity_score

[(0, 0.02831900288102255),
 (1, 0.013642920162872675),
 (2, 0.0),
 (3, 0.0),
 (4, 0.019965150615321666),
 (5, 0.008096335798768265),
 (6, 0.0),
 (7, 0.03432969825299223),
 (8, 0.008072423649825716),
 (9, 0.0),
 (10, 0.023061623071613118),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.02036981316760844),
 (15, 0.0),
 (16, 0.03255185140594444),
 (17, 0.027180828825043545),
 (18, 0.023523549881006252),
 (19, 0.03090983449361584),
 (20, 0.0077831485167801214),
 (21, 0.0),
 (22, 0.03219171052847813),
 (23, 0.0),
 (24, 0.007373257015273083),
 (25, 0.0),
 (26, 0.03543123412429556),
 (27, 0.05253219280786348),
 (28, 0.020505792176068214),
 (29, 0.0),
 (30, 0.015753870497987134),
 (31, 0.06428698874458375),
 (32, 0.0),
 (33, 0.048480898917700695),
 (34, 0.0),
 (35, 0.021590153687098518),
 (36, 0.03117795428732212),
 (37, 0.01299145564925202),
 (38, 0.0),
 (39, 0.019309360684936196),
 (40, 0.008725804870656068),
 (41, 0.019751183328823068),
 (42, 0.004114522971656119),
 (43, 0.020841444156465862),

In [76]:
# All movies score
len(similarity_score)

4803

In [77]:
# Sorting nilai dari terbesar ke kecil
# Sorting berdasarkan similarity scorenya
sorted_similar_travel = sorted(similarity_score, key = lambda x:x[1], reverse = True)
sorted_similar_travel

[(2605, 1.0000000000000002),
 (2826, 0.17925409256006866),
 (2880, 0.16578267302413782),
 (4579, 0.16472122638314324),
 (2389, 0.15996630591785224),
 (3440, 0.15885270831242002),
 (510, 0.15383489602301115),
 (1063, 0.1390105825433066),
 (4401, 0.13768896759270915),
 (4065, 0.13616993425319313),
 (1173, 0.12682615894661356),
 (2084, 0.1267688626741297),
 (1504, 0.12200777438736986),
 (3188, 0.1155502522561574),
 (3702, 0.1123970392064044),
 (322, 0.11143128406226517),
 (1658, 0.11057803685546477),
 (4696, 0.10975707401697515),
 (1485, 0.1094925561980953),
 (525, 0.10719711215100106),
 (1798, 0.10565045438304463),
 (1068, 0.10085018137534506),
 (1585, 0.10046472292307872),
 (96, 0.09974362477953277),
 (2091, 0.09926956625226523),
 (1004, 0.09628854665699409),
 (472, 0.09427712388450459),
 (2529, 0.09347800635621975),
 (2599, 0.09321084321430248),
 (2876, 0.0921535730095686),
 (3324, 0.09131152586221704),
 (1772, 0.09060175868098999),
 (2273, 0.08940623876970877),
 (4121, 0.0887915564263

In [82]:
# Show top 30 travel spots
print('Hasil recomendded travel spot untuk anda: ')

i = 1
for travel in sorted_similar_travel:
    index = travel[0]
    title_from_index = travels_data[travels_data.index == index]['title'].values[0]
    if(i < 30):
        print(str(i) + ".", title_from_index)
        i += 1
    else:
        break;

Hasil recomendded travel spot untuk anda: 
1. Mi America
2. Adore
3. Brazil
4. Rebecca
5. Coco Before Chanel
6. Kill the Messenger
7. Money Train
8. Much Ado About Nothing
9. Austin Powers: The Spy Who Shagged Me
10. A Walk Among the Tombstones
11. Extreme Measures
12. The Horse Whisperer
13. Charlie Bartlett
14. Freeheld
15. Gremlins 2: The New Batch
16. Analyze This
17. Two Weeks Notice
18. Cop Land
19. Below Zero
20. The Skeleton Twins
21. Mickey Blue Eyes
22. The Omega Code
23. In Cold Blood
24. 15 Minutes
25. Home Alone 2: Lost in New York
26. Sea of Love
27. Rampage
28. The Taking of Pelham 1 2 3
29. We Own the Night


In [80]:
# Dijadikan satu code semua
travel_favorite = input("Enter your favorite travel spot: ")
list_of_all_travel_name = travels_data['title'].tolist()
find_close_match = difflib.get_close_matches(travel_favorite, list_of_all_travel_name)
close_match = find_close_match[0]
index_of_travel = travels_data[travels_data.title == close_match]['index'].values[0]
similarity_score = list(enumerate(similarity[index_of_travel]))
sorted_similar_travel = sorted(similarity_score, key = lambda x:x[1], reverse = True)


# Show top 30 travel spots
print('Hasil recomendded travel spot untuk anda: ')

i = 1
for travel in sorted_similar_travel:
    index = travel[0]
    title_from_index = travels_data[travels_data.index == index]['title'].values[0]
    if(i < 30):
        print(str(i) + ".", title_from_index)
        i += 1
    else:
        break;


Hasil recomendded travel spot untuk anda: 
1. Mi America
2. Adore
3. Brazil
4. Rebecca
5. Coco Before Chanel
6. Kill the Messenger
7. Money Train
8. Much Ado About Nothing
9. Austin Powers: The Spy Who Shagged Me
10. A Walk Among the Tombstones
11. Extreme Measures
12. The Horse Whisperer
13. Charlie Bartlett
14. Freeheld
15. Gremlins 2: The New Batch
16. Analyze This
17. Two Weeks Notice
18. Cop Land
19. Below Zero
20. The Skeleton Twins
21. Mickey Blue Eyes
22. The Omega Code
23. In Cold Blood
24. 15 Minutes
25. Home Alone 2: Lost in New York
26. Sea of Love
27. Rampage
28. The Taking of Pelham 1 2 3
29. We Own the Night
